<a href="https://colab.research.google.com/github/0x7o/text2keywords/blob/main/example/keyT5_train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Пример обучения на свою задачу

Мы обучим нашу модель keyT5

In [ ]:
!pip install transformers sentencepiece 

In [ ]:
import pandas as pd
import numpy as np
import torch
from transformers import T5ForConditionalGeneration, T5Tokenizer
import random
from tqdm.auto import tqdm, trange
import os
from sklearn.model_selection import train_test_split

In [ ]:
df = pd.read_csv('train.csv')

In [ ]:
pd.options.display.max_colwidth = 500

In [ ]:
df.sample(5)

Разобьем выборку на обучающую и тестовую, и обучающую превратим в список пар. 

In [ ]:
df_train, df_test = train_test_split(df.dropna(), test_size=0.5, random_state=1)
pairs = df_train[['X', 'Y']].values.tolist()

Инициализируем модель из предобученной

In [ ]:
raw_model = '0x7194633/keyt5-large' # или 0x7194633/keyt5-base
model = T5ForConditionalGeneration.from_pretrained(raw_model).cuda();
tokenizer = T5Tokenizer.from_pretrained(raw_model)

In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=1e-5)

Обучаем модель! Мы пройдём три эпохи, т.е. покажем данные модели по три раза.

На GPU с размером батча 16 одна эпоха должна занять где-то 15 минут. 

In [ ]:
batch_size = 4  # сколько примеров показывем модели за один шаг
report_steps = 200  # раз в сколько шагов печатаем результат
epochs = 3  # сколько раз мы покажем данные модели

In [ ]:
model.train()
losses = []
for epoch in range(epochs):
    print('EPOCH', epoch)
    random.shuffle(pairs)
    for i in trange(0, int(len(pairs) / batch_size)):
        batch = pairs[i * batch_size: (i + 1) * batch_size]
        x = tokenizer([p[0] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y = tokenizer([p[1] for p in batch], return_tensors='pt', padding=True).to(model.device)
        y.input_ids[y.input_ids == 0] = -100
        loss = model(
            input_ids=x.input_ids,
            attention_mask=x.attention_mask,
            labels=y.input_ids,
            decoder_attention_mask=y.attention_mask,
            return_dict=True
        ).loss
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        losses.append(loss.item())
        if i % report_steps == 0:
            print('step', i, 'loss', np.mean(losses[-report_steps:]))

EPOCH 1


  0%|          | 0/4892 [00:00<?, ?it/s]

step 0 loss 3.8470616340637207
step 200 loss 4.236956038475037
step 400 loss 4.243761397600174
step 600 loss 4.185914375782013
step 800 loss 4.21162052154541
step 1000 loss 4.204087338447571
step 1200 loss 4.202062028646469
step 1400 loss 4.189540849924088
step 1600 loss 4.168862791061401
step 1800 loss 4.174042043685913
step 2000 loss 4.123842047452927
step 2200 loss 4.086935578584671
step 2400 loss 4.143192913532257
step 2600 loss 4.176858893632889
step 2800 loss 4.13883326292038
step 3000 loss 4.160567367076874
step 3200 loss 4.092823115587234
step 3400 loss 4.107407578229904
step 3600 loss 4.153625081777573
step 3800 loss 4.086228107213974
step 4000 loss 4.132798237800598
step 4200 loss 4.074807376861572
step 4400 loss 4.124523136615753
step 4600 loss 4.053861576318741
step 4800 loss 4.07164126753807
EPOCH 2


  0%|          | 0/4892 [00:00<?, ?it/s]

step 0 loss 4.070417506694794
step 200 loss 4.070597746372223
step 400 loss 4.094182950258255
step 600 loss 4.063615082502365
step 800 loss 3.967441118955612
step 1000 loss 4.011284437179565
step 1200 loss 4.020660309791565
step 1400 loss 3.9995454287528993
step 1600 loss 3.998213469982147
step 1800 loss 3.971472758054733
step 2000 loss 4.004275197982788
step 2200 loss 3.9787186360359192
step 2400 loss 3.954311252832413
step 2600 loss 3.972656538486481
step 2800 loss 3.9850507843494416
step 3000 loss 3.9864893901348113
step 3200 loss 3.918144265413284
step 3400 loss 3.98400639295578
step 3600 loss 3.972563716173172
step 3800 loss 3.951940791606903
step 4000 loss 3.968209079504013
step 4200 loss 3.967874040603638
step 4400 loss 3.9230076920986177
step 4600 loss 3.9887063550949096
step 4800 loss 3.957128328084946


In [ ]:
model.eval()

def answer(x, **kwargs):
    inputs = tokenizer(x, return_tensors='pt').to(model.device) 
    with torch.no_grad():
        hypotheses = model.generate(**inputs, **kwargs)
    return tokenizer.decode(hypotheses[0], skip_special_tokens=True)

Посмотрим, насколько хорошо модель выучила свою тренировочную выборку. 

In [ ]:
sample = df_train.sample(5)
for i, row in sample.iterrows():
    print(row.X)
    print('real:', row.Y)
    print('model: ', answer(row.X))
    print('---')

Теперь посмотрим, насколько хорошо модель справляется с ответами на вопросы, которые она не видела. 

In [ ]:
sample = df_test.sample(5)
for i, row in sample.iterrows():
    print(row.X)
    print('real:', row.Y)
    print('model: ', answer(row.X))
    print('---')

In [ ]:
for q in ['Тут статья для проверки нашей модели']:
    print(q)
    print(answer(q, do_sample=True, top_p=0.9))
    print()

In [ ]:
new_model_name = 'keyT5-custom'  # название папки
model.save_pretrained(new_model_name)
tokenizer.save_pretrained(new_model_name)